# Hands-on: Generate Summary of Planning Analytics Data 

## Overview

This Jupyter Notebook provides an example of how to:

1. Connect to Planning Analytics server and retrieve dataset from Cube and View as a dataframe using TM1py.

2. Construt a prompt and pass the tabular dataset to Large Language Model (LLM) to generate a summary of the data.

In [ ]:
# Install library
%pip install TM1py

In [ ]:
# Import libraries
import json
import os
import requests

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

from TM1py.Services import TM1Service
from TM1py.Exceptions import TM1pyException

## 1. Setup Connection to Planning Analytics

To get connected to Planning Analytics server, you need these informations:
- address
- port
- user
- password
- namespace

Your connection is successful when you see the *Server Name* and *Product Version* information.

In [ ]:
# Set up connection to Planning Analytics server
try:
    with TM1Service(
        address='<YOUR PA SERVER ADDRESS HERE>,
        port= <YOUR PA PORT NUMBER HERE>,
        ssl=False,
        user="pm",
        password="IBMDem0s",
        namespace='Harmony LDAP'
    ) as tm1:
        print("Server Name:", tm1.server.get_server_name())
        print("Product Version:",tm1.server.get_product_version())

# Error Handling        
except TM1pyException as e:
    if e.status_code == 401:
        print('Wrong credentials')
    elif e.status_code == 404:
        print('Wrong connection')
    else:
        print('Something else went wrong. Check error code:', str(e))

## 2. Retrieve dataset from Cube and View

Define your existing Cube and View name and retrieve it as a dataframe, then convert to markdown.

In [ ]:
# Define Cube and View name to be retrieved
cube_name = 'Revenue'
view_name = 'Input'

In [ ]:
# Retrieve data from the Cube and View as a dataframe
# Reference: https://code.cubewise.com/blog/getting-data-from-tm1-with-python/
df = tm1.cubes.cells.execute_view_dataframe(cube_name=cube_name, 
                                            view_name=view_name, 
                                            private=False)
# Display the dataframe
df

In [ ]:
# Convert dataframe to markdown
df_md = df.to_markdown(index=False)
print(df_md)

## 3. Creating Prompt

Construct your prompt which will be passed to Large Language Model (LLM).

In [ ]:
# Create a prompt
prompt = f"Extract the key findings from the table regarding units sold and describe their development.\n\n{df_md}"
print(prompt)

## 4. Configuring watsonx.ai

The following section defines the input to the Large Language Model (LLM).

Provides the credential for watsonx.ai as indicated below

1. `watsonx_project_id` - The watsonx.ai **Project ID** provided in watsonx.ai project -> Manage -> Project id
2. `api_key` - The **API Key** provided in IBM Cloud -> Manage -> API Key

In [ ]:
# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"

# Replace with your watsonx project id (look up in the project Manage tab)
watsonx_project_id = "<YOUR WATSONX.AI PROJECT ID HERE>"

# Replace with your IBM Cloud key
api_key = "<YOUR IBM CLOUD API KEY HERE>"

In [ ]:
# Initialize the watsonx model
model_init = None

def get_model(model_type, max_tokens, min_tokens, decoding, temperature):#

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature,
    }
    global model_init
    if model_init == None:
        model_init = Model(
            model_id=model_type,
            params=generate_params,
            credentials={
                "apikey": api_key,
                "url": url
            },
            project_id= watsonx_project_id
            )

    return model_init

The following block specifies the parameters for the LLM. In a PoX, you may want to vary these values to show a client how they can get the best results.

1. **model_type** specifies the LLM being used. In the example below it is the llama-2-70b-chat model. You can change it to other models. Note that the size of the model will have implications on resource usage. You may wish to try some of the other LLM in a PoX and see if they will provide different results. Refer [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-api-model-ids.html?context=wx&audience=wdp), for exhaustive list of models supported.

2. **max_tokens** specifies the maximum number of output tokens. Keep in mind that 1 token does not equal 1 word. In general, you can estimate roughly 3 tokens per word.

3. **min_tokens** specifies the minimum number of output tokens.

4. **decoding** specifies the decoding method. You can also choose to do **sampling** decoding - in which case you can specify more parameters (such as **Top-P** and **Top-K**). More information on these additional parameters can be found from the watsonx.ai Technical Sales Level 3 class (https://learn.ibm.com/course/view.php?id=13452).

5. **temperature** specifies how conservative or creative the model will be. The lower it is, the more conservative it it. The range is from 0 to 2.

In [ ]:
# Set up watsonx model and parameters
model_type = "meta-llama/llama-2-70b-chat"
max_tokens = 1000
min_tokens = 50
decoding = DecodingMethods.GREEDY
temperature = 0.7

# Get the watsonx model
model = get_model(model_type, max_tokens, min_tokens, decoding, temperature)

## 5. Summary Generation

This block generates a summary based on the input prompt and the specified parameters.

In [ ]:
# Send a prompt to model
generated_response = model.generate(prompt)
response_text = generated_response['results'][0]['generated_text']

# Print model response
print("--------------------------------- Generated response -----------------------------------")
print(response_text)